# Data Loading

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("../input/ieee-fraud-detection/sample_submission.csv")
test_identity = pd.read_csv("../input/ieee-fraud-detection/test_identity.csv")
test_transaction = pd.read_csv("../input/ieee-fraud-detection/test_transaction.csv")
train_identity = pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")
train_transaction = pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")

In [ ]:
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [ ]:
print(f"Training data contains {train.shape[0]} rows and {train.shape[1]} columns.")
print(f"Test data contains {test.shape[0]} rows and {test.shape[1]} columns.")

In [ ]:
train.head()

In [ ]:
del test_identity, test_transaction, train_identity, train_transaction

# Null values & Same values

In [ ]:
print(f"There are {train.isnull().any().sum()} columns in training data with null values.")
print(f"There are {test.isnull().any().sum()} columns in test data with null values.")

In [ ]:
[c for c in train.columns if train[c].nunique() <= 1]

In [ ]:
[c for c in test.columns if test[c].nunique() <= 1]

# Feature exploration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

## TransactionDT

In [ ]:
train['TransactionDT'].plot(kind='hist',
                                        figsize=(15, 5),
                                        label='train',
                                        bins=50,
                                        title='Train vs Test TransactionDT distribution')
test['TransactionDT'].plot(kind='hist',
                                       label='test',
                                       bins=50)
plt.legend()
plt.show()

## TransactionAMT

In [ ]:
train['TransactionAmt'].plot(
    kind='hist',
    bins=1000,
    figsize=(15, 5),
    xlim=(0,10000),
    label='train',
    title="Distribution of TransactionAmt in linear scale"
)
test['TransactionAmt'].plot(
    kind='hist',
    bins=1000,
    figsize=(15, 5),
    xlim=(0,10000),
    label='test',
)
plt.legend()
plt.show()

In [ ]:
train['TransactionAmt'].apply(np.log) \
    .plot(
        kind='hist',
        bins=100,
        figsize=(15, 5),
        title="Distribution of TransactionAmt in log scale",
        label='train',
    
)
test['TransactionAmt'].apply(np.log) \
    .plot(
        kind='hist',
        bins=100,
        figsize=(15, 5),
        label='test',
)
plt.legend()
plt.show()

In [ ]:
train.groupby("ProductCD")["TransactionID"].count() \
    .plot(kind='barh', title="Number of transactions by ProductCD")

In [ ]:
train.groupby("ProductCD")["isFraud"].mean() \
    .plot(kind='barh', title="Percentage of fraud by ProductCD")

plt.show()

## Card features

card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.

In [ ]:
card_features = [c for c in train.columns if 'card' in c]
card_cols = train[card_features]

In [ ]:
card_cols.head()

In [ ]:
card_cols.nunique()

card4 and card6 seem to be credit card company and card type. One-hot encode?

### Numerical Card features

In [ ]:
color_idx = 0
for c in card_cols:
    if train[c].dtype in ['float64','int64']:
        train[c].plot(kind='hist',
              title=c,
              bins=50,
              figsize=(15, 2),
              color=color_pal[color_idx])
    color_idx += 1
    plt.show()

## Addr features

addr1, addr2 are both categorical features.

In [ ]:
addr_features = [c for c in train.columns if 'addr' in c]
addr_cols = train[addr_features]

In [ ]:
addr_cols.head()

In [ ]:
addr_cols.nunique()